In [1]:
from keras.models import load_model
import numpy as np
import copy
import cv2
import os
import random
from datetime import datetime

Using TensorFlow backend.
/home/aadi/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/aadi/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/aadi/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/aadi/.local/lib/python3.6/site-packages/tensorflow/python/framewo

In [2]:
def binaryMasking(img):
    img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    img = cv2.GaussianBlur(img, (7,7), 3)
    img = cv2.adaptiveThreshold(img, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY_INV, 11, 2)
    ret, new = cv2.threshold(img, 25, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)
    return new

In [3]:
def initalize_class_for_dataset(name):
    global className, count
    className = name
    if not os.path.exists(os.path.join("dataset", className)):
        os.makedirs(os.path.join("dataset", className))
    count = len(os.listdir(os.path.join("dataset", className)))

In [4]:
model = load_model('model.h5')

initalize_class_for_dataset("NONE")

x0, y0, width = 300, 180, 300

# params
current_color = (255, 0, 0)
training = False
font = cv2.FONT_HERSHEY_DUPLEX
text_loc = [10, 50]
count_height = 45
className = "None"
prediction = ""
count = 0
showMask = False
classes = ["ROCK", "PAPER", "SCISSORS", "NONE"]
currentClass = "NONE"
computer_points = 0
your_points = 0
gameStarted = False
time_remaining = 4
startTime = datetime.now()
countdown = True

# setup opencv
cam = cv2.VideoCapture(0)
#cv2.namedWindow('Original', cv2.WINDOW_NORMAL)

while True:
    ret, frame = cam.read()
    
    # get rid of the inverted image display
    #frame = cv2.flip(frame, 1)
    
    # copy to display rectangles and texts and all those stuffs
    window = copy.deepcopy(frame)
    
    
    
    if training:
        if countdown:
            cv2.putText(window, "Starting training in " + str(time_remaining), (text_loc[0], text_loc[1]), font, 0.5, current_color, 1, 1)
            time_elapsed = (datetime.now() - startTime).total_seconds()
            
            cv2.rectangle(window, (x0, y0), (x0 + width - 1, y0 + width - 1), (230, 230, 230), 2)
            
            if time_elapsed >= 1:
                time_remaining -= 1
                startTime = datetime.now()
                
            if time_remaining == 0:
                countdown = False
        else:
            cv2.putText(window, 'Key Bindings :', (50, 25), font, 0.5, (255, 255, 255), 1, 1)
            cv2.putText(window, '1 : Rock, 2 : Paper, 3 : Scissors, 4 : None (Default)', (50, 45), font, 0.5, (190, 190, 190), 1, 1)
            time_remaining = 4
            current_color = (0, 255, 0)
            cv2.putText(window, "Taking training data. . .", (text_loc[0], text_loc[1] + 30), font, 0.5, current_color, 1, 1)
            # the rectangle
            cv2.rectangle(window, (x0, y0), (x0 + width - 1, y0 + width - 1), current_color, 2)
    else:
        current_color = (255, 0, 0)
        cv2.putText(window, "", (text_loc[0], text_loc[1]), font, 1, current_color, 2, 1)
    
    # get the area of interest
    reg_of_interest = frame[y0:y0 + width, x0:x0 + width]
    reg_of_interest = binaryMasking(reg_of_interest)
    
    if showMask:
        window[y0:y0 + width, x0:x0 + width] = cv2.cvtColor(reg_of_interest, cv2.COLOR_GRAY2BGR)
    
    if training and not countdown:
        cv2.putText(window, 'Current class : %s'%(className), (text_loc[0], text_loc[1] + count_height + 20), font, 0.7, 
                (255, 0, 0), 1, 1)
        cv2.putText(window, 'Number of samples takes : %d'%(count), (text_loc[0], text_loc[1] + count_height + 40), font, 0.7, 
                (255, 0, 0), 1, 1)
        cv2.imwrite(os.path.join('dataset', className, str(count) + ".png"), reg_of_interest)
        count+=1
    else:
        # normalize
        img = np.float32(reg_of_interest)/255
        img = np.expand_dims(img, axis = 0)
        img = np.expand_dims(img, axis = -1)
        prediction_model = model.predict(img)[0]
        current_prediction = classes[np.argmax(prediction_model)]
        
        if gameStarted:
            # the rectangle
            cv2.rectangle(window, (x0, y0), (x0 + width - 1, y0 + width - 1), current_color, 7)
            if prediction == "NONE":
                your_points = 0
                computer_points = 0
                prediction = ""
                x = ""
            cv2.putText(window, 'After starting, just move your hand away to restart, or press s to Quit', (text_loc[0], text_loc[1]), font, 0.5, (255, 0, 0), 1, 1)
            cv2.putText(window, 'You : %s ( Points : %s )'%(prediction, str(your_points)), (text_loc[0], text_loc[1] + 60), font, 0.5, (0, 255, 0), 1, 1)
            cv2.putText(window, 'Computer : %s ( Points : %s )'%(x, str(computer_points)), (text_loc[0], text_loc[1] + 90), font, 0.5, (0, 255, 0), 1, 1)
            
        elif training:
            cv2.putText(window, '', (180, 25), font, 0.5, (255, 255, 255), 1, 1)
            cv2.putText(window, '', (180, 45), font, 0.5, (255, 255, 255), 1, 1)
        
        else:
            cv2.putText(window, 'Press T to collect training data', (180, 25), font, 0.5, (255, 255, 255), 1, 1)
            cv2.putText(window, 'Press S start game when ready', (180, 45), font, 0.5, (255, 255, 255), 1, 1)
            
    cv2.imshow("Original", window)
    
    # rock paper scissors
    if current_prediction != prediction:
        
        prediction = current_prediction
        x = random.choice(["ROCK", "PAPER", "SCISSORS"])
        
        if x == "ROCK" and prediction =="SCISSORS":
            computer_points += 1

        if x == "SCISSORS" and prediction =="ROCK":
            your_points += 1

        if x == "ROCK" and prediction =="PAPER":
            your_points += 1

        if x == "PAPER" and prediction =="ROCK":
            computer_points += 1

        if x == "PAPER" and prediction =="SCISSORS":
            your_points += 1

        if x == "SCISSORS" and prediction =="PAPER":
            computer_points += 1
        
        
    
    # keyboard keys
    key = cv2.waitKey(10) & 0xff
    
    if key == ord('q'):
        break
        
    elif key == ord('s'):
        gameStarted = not gameStarted
        
    elif key == ord('m'):
        showMask = not showMask
    
    elif key == ord('t'):
        if training:
            countdown = True
        training = not training
        
    elif key == ord('1'):
        initalize_class_for_dataset("ROCK")
        
    elif key == ord('2'):
        initalize_class_for_dataset("PAPER")
        
    elif key == ord('3'):
        initalize_class_for_dataset("SCISSORS")
    
    elif key == ord('4'):
        initalize_class_for_dataset("NONE")
        

cam.release()
cv2.destroyAllWindows()


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where

